In [1]:
import numpy as np
from scipy.special import softmax
from src.utils.brute_force import BruteForceOptimizer
from src.algorithms.models import MPAssortOriginal, MPAssortSurrogate
from src.utils.distributions import GumBel
from src.utils.lp_optimizers import LinearProgramSolver
import time
from src.algorithms.BB import branch_and_bound


In [2]:
class RSP_obj:
    def __init__(self, model):
        self.model = model

    def __call__(self, w):
        return self.model.RSP(w)[1]

class RSP_ub:
    def __init__(self, model):
        self.model = model

    def __call__(self, box_low, box_high):

        N = len(self.model.u)

        # Compute objective coefficients c
        c = self.model._probs_buying_surrogate(box_low) * self.model.r
        
        # Construct constraint matrix A
        A = np.vstack([
            self.model._probs_U_exceed_w(box_high),  # First |B| rows are P(u[j] + X > w[i])
            np.ones(N),   # Cardinality upper bound
            -np.ones(N)   # Cardinality lower bound
        ])
        
        # Compute RHS vector b
        b = np.concatenate([
            self.model._Bs,
            [self.model.C[1], -self.model.C[0]]
        ])

        lp_solver = LinearProgramSolver()
        upper_bound, _, status = lp_solver.maximize(c, A, b)
        if status != 'Optimal':
            raise ValueError(f"Failed to solve RSP upper bound: {status}")
        return upper_bound
    
class RSP_lb:
    def __init__(self, model):
        self.model = model

    def __call__(self, box_low, box_high):
        box_middle = (box_low + box_high) / 2
        # rsp_box_low = self.model.RSP(box_low)
        # rsp_box_high = self.model.RSP(box_high)
        rsp_box_middle = self.model.RSP(box_middle)[1]
        return rsp_box_middle, box_middle

        # return max(rsp_box_low, rsp_box_middle, rsp_box_high)

#### Model Params

In [3]:
# Problem parameters
N = 15  # Number of products
C = (12, 12)  # Cardinality constraints

# Generate random problem instance
np.random.seed(42)
u = np.random.normal(0, 1, N)
r = np.random.uniform(1, 10, N)

# Generate basket size distribution
basket_sizes = [1, 2, 3]
probs = np.random.normal(0, 1, len(basket_sizes))
probs = softmax(probs)
B = dict(zip(basket_sizes, probs))

# Create distribution
distr = GumBel()

# Create objective functions
n_samples = 10000
op = MPAssortOriginal(u, r, B, distr, C, samples=distr.random_sample((n_samples, len(u)+1)))
sp = MPAssortSurrogate(u, r, B, distr, C)

In [4]:
# box_low = np.array([-5, -5, -5], dtype=float)  # Lower bounds of objective function
# box_high = np.array([5, 5, 5], dtype=float)  # Upper bounds of objective function
w_range = np.array(sp._get_box_constraints())
box_low = np.array(w_range[:, 0]).reshape(-1)
box_high = np.array(w_range[:, 1]).reshape(-1)
print("box_low", np.round(box_low, 2))
print("box_hig", np.round(box_high, 2))

rsp_obj = RSP_obj(sp)
rsp_ub = RSP_ub(sp)
rsp_lb = RSP_lb(sp)

# x = (box_low + box_high) / 2
# print(rsp_ub(box_low, box_high))

# Run branch and bound algorithm
best_solution, best_objective = branch_and_bound(rsp_obj, rsp_lb, rsp_ub, box_low, box_high, tolerance=0.5)

print(f"Optimal solution: {best_solution}")
print(f"Optimal objective value: {best_objective}")

box_low [1.78 1.02 0.55]
box_hig [2.44 1.67 1.18]
Layer= 0, ub=14.9992, lb=11.0990
Layer= 1, ub=14.7304, lb=11.1159
Layer= 2, ub=14.3302, lb=11.1159
Layer= 3, ub=13.7890, lb=11.1176
Layer= 4, ub=13.6757, lb=11.2173
Layer= 5, ub=13.4796, lb=11.3316
Layer= 6, ub=12.4922, lb=11.3364
Layer= 7, ub=12.4922, lb=11.4134
Layer= 8, ub=12.4922, lb=11.4652
Layer= 9, ub=-inf, lb=11.4652
Optimal solution: [2.06859589 1.30630493 0.78536987]
Optimal objective value: 11.465196388914844


In [5]:
from src.utils.brute_force import BruteForceOptimizer


In [6]:
num_cores = 4
bf_optimizer = BruteForceOptimizer(N=N, C=C, num_cores=num_cores)

start_time = time.time()
x_op, val_op = bf_optimizer.maximize(op)
time_op = time.time() - start_time

print(f"Optimal solution: {x_op}")
print(f"Selected indices: {np.where(x_op == 1)[0]}")
print(f"Optimal value: {val_op:.4f}")
print(f"Computation time: {time_op:.4f} seconds")

Optimal solution: [1 1 1 1 1 1 0 1 1 0 1 1 0 1 1]
Selected indices: [ 0  1  2  3  4  5  7  8 10 11 13 14]
Optimal value: 11.1917
Computation time: 4.3863 seconds


In [8]:
print("\n=== RSP with branch-and-bound ===")
start_time = time.time()
w = [2.06859589, 1.30630493, 0.78536987]
x_rsp, _ = sp.SP(w)
time_sp = time.time() - start_time

print(f"Optimal solution: {x_rsp}")
print(f"Selected indices: {np.where(x_rsp == 1)[0]}")
print(f"Optimal value: {op(x_rsp):.4f}")
print(f"Computation time: {time_sp:.4f} seconds")

# Compare solutions under original objective
print("\n=== Solution Comparison under Original Pi ===")
op_val_for_x_op = op(x_op)
op_val_for_x_sp = op(x_rsp)

print(f"Original Pi value for x_op: {op_val_for_x_op:.4f}")
print(f"Original Pi value for x_sp: {op_val_for_x_sp:.4f}")
print(f"Relative gap: {(op_val_for_x_op - op_val_for_x_sp)/op_val_for_x_op:.4%}")


=== RSP with branch-and-bound ===
Optimal solution: [1 1 1 1 1 1 0 1 1 0 1 1 0 1 1]
Selected indices: [ 0  1  2  3  4  5  7  8 10 11 13 14]
Optimal value: 11.1917
Computation time: 0.0354 seconds

=== Solution Comparison under Original Pi ===
Original Pi value for x_op: 11.1917
Original Pi value for x_sp: 11.1917
Relative gap: 0.0000%
